### Load tensorRT graph

In [3]:
import tensorflow as tf
from tensorflow.python.platform import gfile
#GRAPH_PB_PATH = './trained_models_local/saved_for_lab/tf_model_base_1502.pb'
GRAPH_PB_PATH_OD = './frozen_model_od/tf_ssd7_model.pb'
#GRAPH_PB_PATH = './converted_trt_graph/trt_graph_st_prg_1601_80p.pb'
#GRAPH_PB_PATH = './converted_trt_graph/trt_graph_quantized.pb'
#GRAPH_PB_PATH_FROZEN_SS='./frozen_model_ss/frozen_model_ss_plf.pb'
GRAPH_PB_PATH_FROZEN_SS='./converted_trt_graph_ss/trt_graph_ss_model.pb'


'''
#loading the graph for OD
with tf.Session() as sess1:
   print("load graph")
   with gfile.FastGFile(GRAPH_PB_PATH_OD,'rb') as f:
       graph_def1 = tf.GraphDef()
   graph_def1.ParseFromString(f.read())
   sess1.graph.as_default()
   tf.import_graph_def(graph_def1, name='age_model')
   graph_nodes1=[n for n in graph_def1.node]
   names1 = []
   for t in graph_nodes1:
      names1.append(t.name)
#print(names1)

'''
#loading the graph for SS
with tf.Session() as sess2:
   print("load graph")
   with gfile.FastGFile(GRAPH_PB_PATH_FROZEN_SS,'rb') as f:
       graph_def2 = tf.GraphDef()
   graph_def2.ParseFromString(f.read())
   sess2.graph.as_default()
   tf.import_graph_def(graph_def2, name='changed_model')
   graph_nodes2=[n for n in graph_def2.node]
   names2 = []
   for t in graph_nodes2:
      names2.append(t.name)
    # print operations



#print(names1)
print('      *****************************************************************                                ')
print(names2)

   #                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print(names)

load graph
      *****************************************************************                                
['input_1', 'stem_conv/Conv2D/ReadVariableOp', 'stem_bn/ReadVariableOp', 'stem_bn/ReadVariableOp_1', 'stem_bn/FusedBatchNormV3/ReadVariableOp', 'stem_bn/FusedBatchNormV3/ReadVariableOp_1', 'block1a_dwconv/depthwise/ReadVariableOp', 'block1a_bn/ReadVariableOp', 'block1a_bn/ReadVariableOp_1', 'block1a_bn/FusedBatchNormV3/ReadVariableOp', 'block1a_bn/FusedBatchNormV3/ReadVariableOp_1', 'block1a_se_squeeze/Mean-1-DataFormatDimMapNHWCToNCHW-LayoutOptimizer', 'block1a_se_reshape/strided_slice/stack', 'block1a_se_reshape/strided_slice/stack_1', 'block1a_se_reshape/strided_slice/stack_2', 'block1a_se_reshape/Reshape/shape/1', 'block1a_se_reshape/Reshape/shape/2', 'block1a_se_reshape/Reshape/shape/3', 'block1a_se_reduce/Conv2D/ReadVariableOp', 'block1a_se_reduce/BiasAdd/ReadVariableOp', 'block1a_se_expand/Conv2D/ReadVariableOp', 'block1a_se_expand/BiasAdd/ReadVariableOp', 'block1a_

### Loading the pb graph

In [ ]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(graph_def, name='')

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print('done')

### Imports for the y_decoded_pred[]

In [ ]:
import os
os.chdir('/home/agxdbot/github/ssd_keras/')
#goes a directory up and imports the below
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast
#change the directory back to Jetson
os.chdir('/home/agxdbot/github/Inference_OD_SS')

In [ ]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

#tf_sess1 = tf.Session(config=tf_config)
#tf.import_graph_def(graph_def1, name='')


tf_sess2 = tf.Session(config=tf_config)
tf.import_graph_def(graph_def2, name='')

In [ ]:
tf_input1 = tf_sess1.graph.get_tensor_by_name('input_1:0')
print(tf_input1)
tf_predictions1 = tf_sess1.graph.get_tensor_by_name('predictions/concat:0')
print(tf_predictions1)

In [ ]:
tf_input2 = tf_sess2.graph.get_tensor_by_name('input_1:0')
print(tf_input2)

tf_predictions2 = tf_sess2.graph.get_tensor_by_name('sigmoid/Sigmoid:0')
print(tf_predictions2)


### Inference on live camera data with pb graph

In [ ]:
import cv2
import numpy as np
#import matplotlib.pyplot as plt

##Uncomment the below import if you don't want the frames to be updated on jupyter notebook or if you are using opencv frames to visualize
#from IPython.display import clear_output


from tensorflow.python.keras.backend import set_session
graph = tf.get_default_graph()


## Drawing a bounding box around the predictions

classes = ['background', 'car', 'truck', 'pedestrian', 'bicyclist', 'light'] # Just so we can print class names onto the image instead of IDs
font = cv2.FONT_HERSHEY_SIMPLEX
  

# fontScale
fontScale = 0.5
   
# Blue color in BGR
color = (255, 255, 0)
  
# Line thickness of 2 px
thickness = 1


#Capture the video from the camera

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    #frame2 = frame.reshape((300,480))
    image_resized2 = cv2.resize(frame, (480,300))
    
    image_resized3 = cv2.resize(frame, (480,320))

    #print(image_resized2.shape)
    #frame2 = np.expand_dims(image_resized2, axis=0)
    
    #Run the Detections using model.predict

    if ret:
        with graph.as_default():
            set_session(sess)
            inputs1, predictions1 = tf_sess.run([tf_input1, tf_predictions1], feed_dict={
            tf_input: image_resized2[None, ...]
        })
        
        y_pred_decoded = decode_detections(predictions1,
                                   confidence_thresh=0.5,
                                   iou_threshold=0.45,
                                   top_k=200,
                                   normalize_coords=True,
                                   img_height=300,
                                   img_width=480)
        np.set_printoptions(precision=2, suppress=True, linewidth=90)
        
        for box in y_pred_decoded[0]:
            
            xmin = box[-4]
            ymin = box[-3]
            xmax = box[-2]
            ymax = box[-1]
            label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
            #cv2.rectangle(im2, (xmin,ymin),(xmax,ymax), color=color, thickness=2 )
            cv2.rectangle(image_resized2, (int(xmin),int(ymin)),(int(xmax),int(ymax)), color=(0,255,0), thickness=2 )
            cv2.putText(image_resized2, label, (int(xmin), int(ymin)), font, fontScale, color, thickness)
        cv2.imshow('Input Images',image_resized2)
        
        
        ## Semantic Segmentation Inference here
        with graph.as_default():
            set_session(sess)
            inputs2, predictions2 = tf_sess2.run([tf_input2, tf_predictions2], feed_dict={
            tf_input: image_resized3[None, ...]
        })
        #cv2.imwrite('file5.jpeg', 255*predictions.squeeze())
        pred_image = 255*predictions2.squeeze()

        ##converts pred_image to CV_8UC1 format so that ColorMap can be applied on it
        u8 = pred_image.astype(np.uint8)

        #Color map autumn is applied to the CV_8UC1 pred_image
        im_color = cv2.applyColorMap(u8, cv2.COLORMAP_AUTUMN)
        #cv2.imshow('input image', image_resized2)
        cv2.imshow('prediction mask',im_color)

        #cv2.waitKey(0)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    else:
        cap.release()
        break

cap.release()
cv2.destroyAllWindows()